In [1]:
import numpy as np

import torch
import csv
import torch.nn as nn
import torch.nn.functional as F
from sklearn.manifold import TSNE

from sklearn.decomposition import PCA, TruncatedSVD



# Generate the high dimensional features and save them to a file.

In [ ]:
# load network

In [ ]:
# load dataset

In [ ]:
# save features

# Load the high dimensional data and do dimensionality reduction using PCA and TSNE

In [ ]:
# load data
data_fpath = 

In [ ]:

# high_d 为 numpy 格式的高维数据
# TSNE 方法只能降维数据至 3 维（包括）以下，专注可视化效果
low_d = TSNE(random_state=0).fit_transform(high_d) 

# PCA、SVD 方法可以降维数据到高于 3 维
low_d = PCA(random_state=0, n_components=128).fit_transform(high_d) 
low_d = TruncatedSVD(random_state=0, n_components=128).fit_transform(high_d) 
